# Deploying TAS4K8s to TKG standalone on vSphere 6.7U3

* TKG 1.1
* No load balancer
* Uses vsphere CNS through CSI

In [ ]:
#env
export TAS_ROOT=~/Downloads/tas-k8s-beta-workshop-tkg/tanzu-application-service
export TAS_CONFIG_DIR=~/Downloads/tas-k8s-beta-workshop-tkg/configuration-values
export TAS_ARCHIVE=~/Downloads/tanzu-application-service.0.1.0-build.252.tar
mkdir -p $TAS_ROOT $TAS_CONFIG_DIR
cd $TAS_ROOT

In [ ]:
#target the right cluster
kubectl config get-contexts

In [ ]:
kubectl config use-context tkg-guest-admin@tkg-guest

In [ ]:
kubectl get all -o wide --all-namespaces

# Deploy TAS

In [ ]:
#extract TAS downloaded archive
tar -C $TAS_ROOT -xvf $TAS_ARCHIVE 

In [ ]:
#remove resource requirements
curl https://raw.githubusercontent.com/cloudfoundry/cf-for-k8s/master/config-optional/remove-resource-requirements.yml > $TAS_ROOT/custom-overlays/remove-resource-requirements.yml

## Generate config values

In [ ]:
./bin/generate-values.sh -d "tas-tkg.lan" > ../configuration-values/deployment-values.yml

Add entries for Pivnet registry and your own image registry (place at end of file, or in two separate files, system-registry-values.yml and app-registry-values.yml):
```
system_registry:
  hostname: registry.pivotal.io
  username: pivnet-username
  password: password

app_registry:
  hostname: https://index.docker.io/v1/
  repository: "username"
  username: "uername"
  password: "password"
```

In [ ]:
#Copy my secrets from pre-existing location
cp ~/Documents/app-registry-values.yml $TAS_CONFIG_DIR
cp ~/Documents/system-registry-values.yml $TAS_CONFIG_DIR

## Default storage class

In [ ]:
kubectl get storageclass

TKG uses CNS through the CSI provisioner. Define your default storage class in a new file config-optional/vsphere-csi-storage-class.yaml:

```
---
kind: StorageClass
apiVersion: storage.k8s.io/v1
metadata:
  name: thin
  annotations:
    storageclass.kubernetes.io/is-default-class: "true"
provisioner: csi.vsphere.vmware.com
parameters:
  storagepolicyname: "k8s-storage-policy"
  fstype: ext4
```

Perform additional steps in vcenter:
* Create new category under "tags and custom attributes" assigned to object types Folder, Datastore, Datastore Cluster. Name it "k8s-storage"
* Pick your datastore -> summary -> Tags, Assign...
* Add tag: k8s-storage-mydatastore, pick k8s-storage from category dropdown
* Create a vm storage policy named "k8s-storage-policy", enable tag based placement rules, add tag rule for k8s-storage category, use storage tagged with, browse tags and select k8s-storage-mydatastore. The desired datastore should appear as compatible.


In [ ]:
kapp deploy -a default-storage-class -f config-optional/vsphere-csi-storage-class.yaml -y

In [ ]:
#deploy tas
./bin/install-tas.sh ../configuration-values

### Configure DNS to external IPs of worker nodes

In [ ]:
kubectl get nodes --output='wide'

# Tear down environment

In [ ]:
#remove cf app from cluster
kapp delete -a cf -y